In [62]:
#Paper implementation

In [63]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import imageio
from albumentations import VerticalFlip, HorizontalFlip, ElasticTransform, GridDistortion, OpticalDistortion, CoarseDropout, ShiftScaleRotate, Compose

In [64]:
# takes a path and creates a directory if it does not exist
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [65]:
def load_data(path):
    """X = images and Y = masks """
    train_x = sorted(glob(os.path.join(path, "train", "image", "*.*")))
    train_y = sorted(glob(os.path.join(path, "train", "mask", "*.*")))

    test_x = sorted(glob(os.path.join(path, "test", "image", "*.*")))
    test_y = sorted(glob(os.path.join(path, "test", "mask", "*.*")))

    return (train_x, train_y), (test_x, test_y)

In [72]:
def augment_data(images, masks, save_path, augment=True):
    H = 512
    W = 512

    for idx, (x, y) in tqdm(enumerate(zip(images, masks)), total=len(images)):
        """Extracting names"""
        name = x.split("\\")[-1].split(".")[0]

        """Reading image and mask"""
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        y = cv2.imread(y, cv2.IMREAD_COLOR)

        if augment:
            
            #Horizontal Flip and Translate
            aug = HorizontalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x1 = augmented["image"]
            y1 = augmented["mask"]
            aug = Compose([ShiftScaleRotate (rotate_limit=0, border_mode=cv2.BORDER_CONSTANT, shift_limit_x=-0.2, shift_limit_y=0.2, p=1)])
            augmented = aug(image=x1, mask=y1)
            x1 = augmented['image']
            y1 = augmented['mask']
 
            #Vertical Flip and Translate
            aug = VerticalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x2 = augmented["image"]
            y2 = augmented["mask"]
            aug = Compose([ShiftScaleRotate (rotate_limit=0, border_mode=cv2.BORDER_CONSTANT, shift_limit_x=-0.15, shift_limit_y=0.15, p=1)])
            augmented = aug(image=x2, mask=y2)
            x2 = augmented['image']
            y2 = augmented['mask']

            #Only X-Y translation
            aug = Compose([ShiftScaleRotate (rotate_limit=0, border_mode=cv2.BORDER_CONSTANT, shift_limit_x=-0.1, shift_limit_y=0.1, p=1)])
            augmented = aug(image=x, mask=y)
            x3 = augmented['image']
            y3 = augmented['mask']

            X = [x, x1, x2, x3]
            Y = [y, y1, y2, y3]

        else:
            X = [x]
            Y = [y]

        index = 0
        for image, mask in zip(X, Y):
            image = cv2.resize(image, (W, H))
            mask = cv2.resize(mask, (W, H))

            if len(X) == 1:
                tmp_image_name = f"{name}.jpg"
                tmp_mask_name = f"{name}.jpg"
            else:
                tmp_image_name = f"{name}_{index}.jpg"
                tmp_mask_name = f"{name}_{index}.jpg"

            image_path = os.path.join(save_path, "image", tmp_image_name)
            mask_path = os.path.join(save_path, "mask", tmp_mask_name)

            """Saving the image"""
            cv2.imwrite(image_path, image)
            cv2.imwrite(mask_path, mask)

            index += 1

In [73]:
if __name__ == "__main__":
    """Seeding"""
    np.random.seed(42)

    """Load the data"""
    data_path = "Data/"
    (train_x, train_y) = load_data(data_path)
    print(f"Train: {len(train_x)} - {len(train_y)}")
    
    """DATA AUGMENTATION"""
    # Creating directories
    create_dir("Phase_3/Train/Image")
    create_dir("Phase_3/Train/Mask")

    augment_data(train_x, train_y, "Phase_1/Train/", augment=True)

Train: 1752 - 1752


100%|██████████| 1752/1752 [02:14<00:00, 13.04it/s]
